In [1]:
import spacy
import pandas as pd
from spacy import displacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

nlp = spacy.load('pt')

In [25]:
doc = nlp(r'Quais agentes de doenças podem ser transmitidos aos bovinos pelos carrapatos?')
#[token for token in doc if (token.pos_ == 'NOUN' or token.pos_ =='VERB')]
#displacy.render(doc, style='dep', jupyter=True)
set([(token, token.dep_, token.head.text) for token in doc if (token.pos_ in ['VERB', "NOUN", "ADJ", 'ADV'] and token.dep_ in ['obj', 'nmod', 'amod', 'compound'])])

{(doenças, 'nmod', 'agentes'),
 (pelos, 'amod', 'bovinos'),
 (carrapatos, 'amod', 'bovinos')}

In [2]:
corpus = pd.read_csv('data/Raw/quinhentas_perguntas.csv', sep=',', header=0)
answers = pd.read_csv('data/Raw/quinhentas_respostas.csv', sep=",", header=0)
corpus['resposta'] = answers['resposta']

In [6]:
relations = set()
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
for i, row in corpus.iterrows():
    text = row['pergunta']+" "+row['resposta']
    doc = nlp(text)
    #relations.update((lemmatizer(token.orth_, token.pos_)[0], token.dep_, lemmatizer(token.head.text, token.head.pos_)[0]) for token in doc if (token.pos_ in ['VERB', "NOUN", "ADJ", 'ADV'] and token.dep_ in ['obj', 'nsubj', 'iobj', 'obl', 'amod', 'compound', 'conj', 'flat']))
    for token in doc:
        tLemma = lemmatizer(token.orth_, token.pos_)[0]
        if not tLemma in topterms:
            continue
        descendants = (w for w in token.children if w.pos_ in ["VERB", "NOUN", "ADJ"])
        for children in descendants:
            cLemma = lemmatizer(children.orth_, children.pos_)[0]
            if not cLemma in topterms:
                continue
            if children.dep_ in ['obj', 'nsubj', 'iobj']:
                relations.add((tLemma, children.dep_, cLemma))
                continue
            if children.dep_ == 'conj' and children.pos_==token.pos_=='NOUN':
                relations.add((tLemma, children.dep_, cLemma))
                continue
            if token.pos_ == "NOUN" and children.pos_ == "NOUN" and children.dep_ in ['flat', 'compound']:
                relations.add((tLemma, children.dep_, cLemma))
                continue
            if token.pos_ == "NOUN" and children.pos_ == "ADJ" and children.dep_ == "amod":
                relations.add((tLemma, children.dep_, cLemma))
                continue
            if token.pos_ == "NOUN" and children.pos_ == "VERB" and children.dep_ == "obl":
                relations.add((tLemma, children.dep_, cLemma))

            


In [12]:
[w for w in relations if w[1]=="compound"]

[('benefício', 'compound', 'heterose'),
 ('início', 'compound', 'manhã'),
 ('mercado', 'compound', 'produto')]

In [4]:
import sys
sys.path.append("./src/")
from CorpusProcessingTools.terms import TermExtractor
#df = corpus
#df[df.columns[1]] = df[df.columns[1]] + df[df.columns[2]]
topterms = TermExtractor.getTopTerms(df, 2600, 'pt', True)



In [15]:
terms = set(topterms)
terms = list(terms)
tdf = pd.DataFrame(terms)

In [16]:
tdf.to_csv('data/TreatedTerms/tfidfterms.csv')